In [5]:
import os
import sys
sys.path.insert(1, '../../Src/')
import numpy as np
from utils import preprocessing
from models import training
from models import models
from utils import make_report

PATH_DATA          = '../../Data/'
PATH_MODEL         = '../../Results/Models/'
PATH_RESULTS = '../../Results/'


FNMAME             = "stringer_spontaneous.npy"
FEATURES           = "sresp"


THRESHOLDING_FLAG  = False
THRESHOLDING_MAX   = 95  # 95%


NORMALIZATION_FLAG = True
TEST_SIZE          = 0.2

PCA_FLAG           = True



N_SPLITS = 5


np.random.seed(42)

%matplotlib inline
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
targets = ["run"]#,"pupilArea","pupilCOM_x","pupilCOM_y"]  #  
variances = [0.75]#[0.25,0.5,0.75,0.9]

In [7]:
######################
for TARGET in targets:
    print('*'*20, TARGET,'*'*20)
    # make processing
    X,y = preprocessing.get_dataset(PATH_DATA,FNMAME,FEATURES,TARGET)

    X_train,X_test,y_train,y_test = preprocessing.simple_temporal_splitting(X,y,test_size=TEST_SIZE)
    
    y_train = y_train.astype(np.float32)
    
    mean_train = np.mean(y_train[1-np.isnan(y_train)] )
    
    if np.sum(np.isnan(y_train))>0:
        print(' nulls in y_train target :',np.sum(np.isnan(y_train)))
        y_train[np.isnan(y_train)] = mean_train

    if np.sum(np.isnan(y_test))>0:
        print(' nulls in y_test target :',np.sum(np.isnan(y_test)))
        y_test[np.isnan(y_test)]   = mean_train


    if THRESHOLDING_FLAG:
        X_train,X_test = preprocessing.thresholding(X_train,X_test,thresholing_max=THRESHOLDING_MAX)

    if NORMALIZATION_FLAG:
        X_train,X_test = preprocessing.normalization(X_train,X_test)
    

    for MAX_VARIANCE_EXPLAINED in variances:
        
        if PCA_FLAG:
            #MAX_VARIANCE_EXPLAINED
            X_train_pca,X_test_pca = preprocessing.auto_PCA(X_train,X_test,max_variance_explanation=None,max_components=20)
            #X_train_pca,X_test_pca = preprocessing.auto_PCA(X_train,X_test,max_variance_explanation=MAX_VARIANCE_EXPLAINED)
        
        N_DIMENSION = X_train_pca.shape[1]
        
        training.training_model_cv(X_train_pca,y_train,
                                          X_test_pca,  y_test,
                                          n_splits = N_SPLITS,
                                          total_models=models.total_models,
                                          path_model=PATH_MODEL)
        print('report starting..')        
        make_report.make_report(TARGET,PATH_RESULTS,PATH_MODEL,SUFFIX='dim_'+str(N_DIMENSION)+'_var_'+str(MAX_VARIANCE_EXPLAINED))
        print('TARGET = ',TARGET,'MAX_VARIANCE_EXPLAINED = ',MAX_VARIANCE_EXPLAINED,' report completed!')

******************** run ********************
dict_keys(['sresp', 'run', 'beh_svd_time', 'beh_svd_mask', 'stat', 'pupilArea', 'pupilCOM', 'xyz'])
 X size : (7018, 11983)
 y size : (7018, 1)
X train :  (5264, 11983) y train : (5264, 1)
X test  :  (1754, 11983) y test  : (1754, 1)
Normalization process ..
X train : mean [0]= 6.6743774 std [0]= 6.6743774
X test  : mean [0]= 7.5208654 std [0]= 8.151653
Normalization completed!
X train : mean [0]= 1.9628543e-07 std [0]= 1.9628543e-07
X test  : mean [0]= 0.1156612 std [0]= 1.1138103
PCA processing ..
X_train size : (5264, 11983)
X_test size  : (1754, 11983)
# principal components : 5264
PCA completed!
max variance explained : 1.0
X_train_pca size : (5264, 20)
X_test_pca size  : (1754, 20)
**********  Training  LinearRegression **********
*****  Fold  1 *****
Train fold : RMSE = 3.84851 R2 = 0.78171
Val  fold  : RMSE = 3.77789 R2 = 0.77291
Test total : RMSE = 4.63143 R2 = 0.81007
Saving model ...
Model saved !  in  ../../Results/Models/Linear

/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/sklear

******************************
Saving folds report in  ../../Results/Models/BayesianRidge/BayesianRidge_results_by_fold.csv
Saving summary cv report in  ../../Results/Models/BayesianRidge/BayesianRidge_results.csv
**********  Training  Ridge **********
*****  Fold  1 *****
Train fold : RMSE = 3.84852 R2 = 0.78171
Val  fold  : RMSE = 3.77793 R2 = 0.7729
Test total : RMSE = 4.63154 R2 = 0.81006
Saving model ...
Model saved !  in  ../../Results/Models/Ridge/Ridge_fold_1.pkl
*****  Fold  2 *****
Train fold : RMSE = 3.86373 R2 = 0.7803
Val  fold  : RMSE = 3.72119 R2 = 0.77869
Test total : RMSE = 4.63343 R2 = 0.80991
Saving model ...
Model saved !  in  ../../Results/Models/Ridge/Ridge_fold_2.pkl
*****  Fold  3 *****
Train fold : RMSE = 3.70824 R2 = 0.78608
Val  fold  : RMSE = 4.32985 R2 = 0.75694
Test total : RMSE = 4.68109 R2 = 0.80598
Saving model ...
Model saved !  in  ../../Results/Models/Ridge/Ridge_fold_3.pkl
*****  Fold  4 *****
Train fold : RMSE = 3.84775 R2 = 0.77721
Val  fold  : RM

/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/sklear

******************************
Saving folds report in  ../../Results/Models/PoissonRegressor/PoissonRegressor_results_by_fold.csv
Saving summary cv report in  ../../Results/Models/PoissonRegressor/PoissonRegressor_results.csv
**********  Training  DecisionTreeRegressor **********
*****  Fold  1 *****
Train fold : RMSE = 0.07987 R2 = 0.99991
Val  fold  : RMSE = 4.00723 R2 = 0.7445
Test total : RMSE = 5.30823 R2 = 0.75051
Saving model ...
Model saved !  in  ../../Results/Models/DecisionTreeRegressor/DecisionTreeRegressor_fold_1.pkl
*****  Fold  2 *****
Train fold : RMSE = 0.03295 R2 = 0.99998
Val  fold  : RMSE = 4.28095 R2 = 0.7071
Test total : RMSE = 5.22478 R2 = 0.75829
Saving model ...
Model saved !  in  ../../Results/Models/DecisionTreeRegressor/DecisionTreeRegressor_fold_2.pkl
*****  Fold  3 *****
Train fold : RMSE = 0.05302 R2 = 0.99996
Val  fold  : RMSE = 4.74337 R2 = 0.70829
Test total : RMSE = 5.39705 R2 = 0.74209
Saving model ...
Model saved !  in  ../../Results/Models/Decision

...

Model saved !  in  ../../Results/Models/LGBMRegressor/LGBMRegressor_fold_2.pkl
*****  Fold  3 *****
[50]	training's rmse: 3.44772	training's l2: 11.8868	valid_1's rmse: 4.27787	valid_1's l2: 18.3001
[100]	training's rmse: 2.97843	training's l2: 8.87103	valid_1's rmse: 3.94163	valid_1's l2: 15.5365
[150]	training's rmse: 2.7627	training's l2: 7.63253	valid_1's rmse: 3.80372	valid_1's l2: 14.4683
[200]	training's rmse: 2.61972	training's l2: 6.86293	valid_1's rmse: 3.73229	valid_1's l2: 13.93
[250]	training's rmse: 2.51102	training's l2: 6.30524	valid_1's rmse: 3.6899	valid_1's l2: 13.6154
[300]	training's rmse: 2.41721	training's l2: 5.84292	valid_1's rmse: 3.65655	valid_1's l2: 13.3703
[350]	training's rmse: 2.33553	training's l2: 5.45468	valid_1's rmse: 3.63364	valid_1's l2: 13.2034


/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/lightgbm/basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[400]	training's rmse: 2.2577	training's l2: 5.09721	valid_1's rmse: 3.61546	valid_1's l2: 13.0715
Train fold : RMSE = 2.24566 R2 = 0.92155
Val  fold  : RMSE = 3.59439 R2 = 0.8325
Test total : RMSE = 3.64554 R2 = 0.88232
Saving model ...
Model saved !  in  ../../Results/Models/LGBMRegressor/LGBMRegressor_fold_3.pkl
*****  Fold  4 *****
[50]	training's rmse: 3.55454	training's l2: 12.6347	valid_1's rmse: 3.65545	valid_1's l2: 13.3623
[100]	training's rmse: 3.13238	training's l2: 9.81177	valid_1's rmse: 3.30007	valid_1's l2: 10.8905
[150]	training's rmse: 2.923	training's l2: 8.54391	valid_1's rmse: 3.16401	valid_1's l2: 10.0109
[200]	training's rmse: 2.77145	training's l2: 7.68093	valid_1's rmse: 3.07077	valid_1's l2: 9.42964
[250]	training's rmse: 2.65608	training's l2: 7.05477	valid_1's rmse: 3.02757	valid_1's l2: 9.16618
Train fold : RMSE = 2.58731 R2 = 0.89926
Val  fold  : RMSE = 2.99263 R2 = 0.86923
Test total : RMSE = 3.67963 R2 = 0.88011
Saving model ...
Model saved !  in  ../../

/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'v

[50]	training's rmse: 3.56425	training's l2: 12.7039	valid_1's rmse: 3.52518	valid_1's l2: 12.4269
[100]	training's rmse: 3.10311	training's l2: 9.62926	valid_1's rmse: 3.26801	valid_1's l2: 10.6799
[150]	training's rmse: 2.90296	training's l2: 8.42718	valid_1's rmse: 3.16477	valid_1's l2: 10.0158
[200]	training's rmse: 2.75565	training's l2: 7.59362	valid_1's rmse: 3.09797	valid_1's l2: 9.59742
[250]	training's rmse: 2.63935	training's l2: 6.96618	valid_1's rmse: 3.05809	valid_1's l2: 9.35193
Train fold : RMSE = 2.63163 R2 = 0.89791
Val  fold  : RMSE = 3.02871 R2 = 0.85443
Test total : RMSE = 3.67707 R2 = 0.88028
Saving model ...
Model saved !  in  ../../Results/Models/LGBMRegressor/LGBMRegressor_fold_5.pkl
Train mean cv : RMSE = 2.51467 |  R2 =0.90531
Val   mean cv : RMSE = 3.13143 |  R2 =0.853
Test  mean cv : RMSE = 3.62744 |  R2 =0.88349
Saving plot in  ../../Results/Models/LGBMRegressor/LGBMRegressor_test_prediction.png
******************************
Saving folds report in  ../../